In [4]:
import pandas as pd
import numpy as np
import math
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from IPython.core.display import display

In [6]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [8]:
SEED = 42
target = 'price'

In [9]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [10]:
""" функция перобразования списка в датафрейм (для красоты отображения на экран)"""
def list_cut(to_cut):
    """
    :param to_cut: список для преобразорвания в датафрейм
    :return: в результате на экран выводится таблица
    """

    # Объявим функции для разбиение списка на список стеков и генерировнаия названий для датафрейма
    to_cut = list(np.sort(to_cut))
    cut_list = lambda item: [item[i:i+7] for i in range(0, len(item), 7)]
    name_list = lambda nom: ["vectors"+ str(i) for i in range(0,math.ceil(nom/7))]

    # создание датафрейма и присовение названий векторам (пропуски заполняем '')
    df_ = pd.DataFrame(cut_list(to_cut)).T
    df_.columns = name_list(len(to_cut))
    df_.fillna('', inplace=True)

    # выводим на экран
    display(df_)

In [11]:
""" Функция проверки корреляции с целевым вектором в виде графика heatmap """
def heatmap(drop_list, dd, target = target):
    """
    :param drop_list: список для удаления из датасета
    :param dd: название датасета, по умолчанию = train
    :param target: целевой вектор, по умолчанию значение переменной target извне функции
    :return: выводит только информацию в виде таблицы исключенных столбцов и графика корреляции
    """
    dlst = list(set(dd.columns) - set(drop_list) - {target})
    dlst = list(np.sort(dlst))

    dlst.append(target)
    sns.set(font_scale=1)
    plt.subplots(figsize=(16, 16))
    sns.heatmap(dd[dlst].corr(), square=True,
                annot=True, fmt=".3f", linewidths=0.1, cmap="RdBu", vmin=-1, vmax=1)

    # для удобства отображения делим веткор для модели на части по 7 строк и перобразум в таблицу.
    list_cut(drop_list)

In [22]:
path = '/Users/irenaradzevich/Documents/sergei_dev/ project_05/'

In [23]:
auto = pd.read_csv(path+'all_auto_ru_09_09_2020.csv')

In [11]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
print(f'Train data shape : {train.shape},\n auto data shape : {auto.shape},\n test data shape : {test.shape}')

Train data shape : (103794, 33),
 auto data shape : (89378, 26),
 test data shape : (34686, 32)


---
# 0. Crearing data
---

In [12]:
# drop duplicates and records with zero-prices and records on new cars (they has different webpage structure)
train.drop_duplicates(inplace=True)
train.dropna(subset=['price', 'bodyType'], inplace=True)
print(f'Records {train.shape[0]}, duplicates {train.duplicated().sum()}')

# statistics of NAN in train dataset
nulls = train.isnull().sum()
print('Number of empty features values in train dataset:')
nulls[nulls.values >0]

Records 87764, duplicates 0
Number of empty features values in train dataset:


complectation_dict         61
image                       1
modelDate               87764
model_info              87764
numberOfDoors              61
vehicleConfiguration    87764
vendor                  87764
Владельцы                   7
Владение                62123
ПТС                         1
Состояние                   7
Таможня                    11
dtype: int64

In [13]:
# fill empty 'numberOfDoors' with value as the same bodyType has
doors = dict(train.groupby(['bodyType', 'numberOfDoors'])['numberOfDoors'].count().index)
train['numberOfDoors']=train.bodyType.map(doors)

# fill empty 'complectation_dict' with empty dict
train.complectation_dict = train.complectation_dict.apply(lambda q: dict() if pd.isna(q) else q)

In [14]:
train.numberOfDoors = train.numberOfDoors.apply(int)
train.parsing_unixtime = train.parsing_unixtime.apply(int)
train.productionDate = train.productionDate.apply(int)
train.sell_id = train.sell_id.apply(int)

In [15]:
train.mileage = train.mileage.apply(int)

In [16]:
nulls = train.isnull().sum()
print('Number of empty features values in train dataset:')
nulls[nulls.values >0]

Number of empty features values in train dataset:


image                       1
modelDate               87764
model_info              87764
vehicleConfiguration    87764
vendor                  87764
Владельцы                   7
Владение                62123
ПТС                         1
Состояние                   7
Таможня                    11
dtype: int64

In [17]:
test.isnull().sum()

bodyType                    0
brand                       0
car_url                     0
color                       0
complectation_dict      28268
description                 0
engineDisplacement          0
enginePower                 0
equipment_dict           9996
fuelType                    0
image                       0
mileage                     0
modelDate                   0
model_info                  0
model_name                  0
name                        0
numberOfDoors               0
parsing_unixtime            0
priceCurrency               0
productionDate              0
sell_id                     0
super_gen                   0
vehicleConfiguration        0
vehicleTransmission         0
vendor                      0
Владельцы                   0
Владение                22691
ПТС                         1
Привод                      0
Руль                        0
Состояние                   0
Таможня                     0
dtype: int64

---
### Checking 'test' and 'auto' datadset.
It is very important to understand when they was parsed. The prices in Russian is always run up,
so ML-model can be good to predict today prices, but several month ago parsed test dataset - no,
because only the another level of prices.

In [18]:
auto.price.mean(), train.price.mean() , train.price.mean()/auto.price.mean()

(1294586.3563303659, 1555948.3935326557, 1.201888453345937)

In [19]:
import time
print(time.ctime(test.parsing_unixtime.min()))
print(time.ctime(test.parsing_unixtime.max()))

Mon Oct 19 14:35:06 2020
Mon Oct 26 14:04:24 2020


### Resume:
test data was parsed nearly when was pased auto dataset - in september 2020.
Today's prises in my train dataset is higher on 20.188%.
So we must to divide current prices by the factor 1.3245.

In [20]:
train['price_current']=train.price
train.price = train.price / 1.201888

----
# Clearin data


## bodyType

In [21]:
def vector_items(vector: str):

    train_set = set(train[vector])
    test_set = set(test[vector].unique())
    train_dif = train_set - test_set
    test_dif = test_set-train_set
    print(f'Statistics of items in vector: {vector}.\n ---------------------------')
    print(f'Train has {len(train_set)} unique items and has difference: {train_dif},')
    print('--')
    print(f'Test has {len(test_set)} unique items and has difference: {test_dif}.')
    print(test_set)

In [22]:
vector_items('bodyType')

Statistics of items in vector: bodyType.
 ---------------------------
Train has 23 unique items and has difference: set(),
--
Test has 24 unique items and has difference: {'фастбек'}.
{'хэтчбек 3 дв.', 'внедорожник 3 дв.', 'лифтбек', 'фастбек', 'компактвэн', 'внедорожник открытый', 'хэтчбек 5 дв.', 'внедорожник 5 дв.', 'купе-хардтоп', 'минивэн', 'седан-хардтоп', 'универсал 5 дв.', 'пикап полуторная кабина', 'фургон', 'кабриолет', 'микровэн', 'лимузин', 'тарга', 'седан 2 дв.', 'седан', 'пикап одинарная кабина', 'родстер', 'купе', 'пикап двойная кабина'}


## brand, model_name, model (new vector)

In [23]:
vector_items('brand')

Statistics of items in vector: brand.
 ---------------------------
Train has 12 unique items and has difference: set(),
--
Test has 12 unique items and has difference: set().
{'MITSUBISHI', 'LEXUS', 'MERCEDES', 'NISSAN', 'HONDA', 'SKODA', 'AUDI', 'INFINITI', 'TOYOTA', 'VOLVO', 'BMW', 'VOLKSWAGEN'}


In [24]:
train.model_name = train.model_name.str.upper()

In [25]:
train[train.model_name == 'S-КЛАСС']

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price,price_current
41563,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",Авто новое пробег 2500км в самой топовой компл...,2.9 л,249 л.с.,"{'e-adjustment-wheel': True, 'multi-wheel': Tr...",Дизель,...,NaN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.310438e+07,15750000.0
41564,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",Автомобиль приобретён у официального дилера Па...,3.0 л,367 л.с.,"{'asr': True, 'roller-blind-for-rear-window': ...",Бензин,...,NaN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.318758e+07,15850000.0
41565,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",ЛУКАВТО официальный дилер Mercedes Benz в Рос...,2.9 л,249 л.с.,"{'asr': True, 'roller-blind-for-rear-window': ...",Дизель,...,NaN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.331156e+07,15999000.0
41566,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",Внимание Только для клиентов AVILON Автомобили...,3.0 л,367 л.с.,"{'cruise-control': True, 'asr': True, 'roller-...",Бензин,...,NaN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.329575e+07,15980000.0
41567,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Ventilated disc', 'feeding': '...",В пленке самая полная комплектация нет только ...,3.0 л,367 л.с.,"{'asr': True, 'tinted-glass': True, 'roller-bl...",Бензин,...,NaN,1 владелец,8 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1.289638e+07,15500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43053,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Disc', 'feeding': 'None', 'ful...",На пружинах описание на фото Торг,3.2 л,224 л.с.,"{'tinted-glass': True, 'roller-blind-for-rear-...",Бензин,...,NaN,3 или более,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1.664049e+05,200000.0
43055,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,синий,"{'back-brake': 'Disc', 'feeding': 'None', 'ful...",Автомобиль продается по генеральной довереннос...,3.0 л,180 л.с.,"{'passenger-seat-updown': True, '16-inch-wheel...",Бензин,...,NaN,3 или более,4 года,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1.664049e+05,200000.0
43056,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,синий,"{'back-brake': 'Disc', 'feeding': 'None', 'hor...",Дорест Из плюсов люк работает ходовка мягенька...,2.8 л,156 л.с.,{'hatch': True},Бензин,...,NaN,3 или более,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1.664049e+05,200000.0
43057,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{'back-brake': 'Disc', 'feeding': 'None', 'ful...",БЕЗ ТОРГА По поводу торга не звонить и не писа...,4.3 л,279 л.с.,"{'asr': True, 'tinted-glass': True, 'esp': Tru...",Бензин,...,NaN,3 или более,8 месяцев,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1.664040e+05,199999.0


In [26]:
test[test.model_name == 'S_KLASSE']

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
15416,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,синий,NaN,Позвоните нам и получите дополнительную скидку...,5.5 LTR,388 N12,"{""electro-window-back"":true,""alloy-wheel-disks...",бензин,...,SEDAN AUTOMATIC 5.5,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
15449,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,В ДТП не участвовала.\n\n。Подушка безопасности...,4.7 LTR,435 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",бензин,...,SEDAN AUTOMATIC 4.7,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
15453,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,ЛОТ: 01200668\nFAVORIT MOTORS KIA Юг\n\nВы мож...,3.0 LTR,235 N12,"{""asr"":true,""tinted-glass"":true,""esp"":true,""ad...",дизель,...,SEDAN AUTOMATIC 3.0,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
15492,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,"{""id"":""21016733"",""name"":""S 450 4MATIC"",""availa...",МБ-Беляево — официальный дилер «Мерседес-Бенц»...,3.0 LTR,367 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,SEDAN AUTOMATIC 3.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
15497,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Автодилер года 2020\n\nПобедитель в номинации ...,4.7 LTR,435 N12,"{""cruise-control"":true,""airbag-rear-side"":true...",бензин,...,SEDAN AUTOMATIC 4.7,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34214,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Автомобиль в отличном состоянии.Обслуживался в...,5.0 LTR,306 N12,NaN,бензин,...,SEDAN AUTOMATIC 5.0,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
34226,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Автомобиль Приобретался у официального дилера ...,4.0 LTR,469 N12,NaN,бензин,...,SEDAN AUTOMATIC 4.0,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
34239,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,"Продаётся легендарный S500 короткая база, полн...",5.0 LTR,306 N12,NaN,бензин,...,SEDAN AUTOMATIC 5.0,автоматическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
34241,седан,MERCEDES,https://auto.ru/cars/used/sale/mercedes/s_klas...,чёрный,NaN,Автомобиль продается официальным дилером Автом...,3.0 LTR,235 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",дизель,...,SEDAN AUTOMATIC 3.0,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [27]:
vector_items('model_name')

Statistics of items in vector: model_name.
 ---------------------------
Train has 528 unique items and has difference: {'RS 3', '1 СЕРИИ', '5 СЕРИИ', 'GLA AMG', 'PAJERO MINI', 'POLO GTI', 'GLE AMG', 'GOLF PLUS', 'SPRINTER CARIB', 'GLC COUPE', 'MAYBACH S-КЛАСС', 'SPACE STAR', '190 (W201)', 'GLA', 'G-КЛАСС AMG', 'ALMERA TINO', 'V40 CROSS COUNTRY', 'SCEPTER', 'MR-S', 'CR-X', 'ODYSSEY (NORTH AMERICA)', '7 СЕРИИ', 'COROLLA RUMION', 'LAND CRUISER PRADO', 'PAJERO SPORT', 'TERRANO REGULUS', 'GLC', '1M', 'I-MIEV', 'PRIUS V (+)', 'CLS', 'PRIUS ALPHA', 'MASTERACE SURF', 'FJ CRUISER', 'S-КЛАСС AMG', 'S60 CROSS COUNTRY', 'GLC COUPE AMG', 'GRAND HIACE', 'GLE', "R'NESSA", 'COROLLA II', 'GOLF GTI', 'CIVIC TYPE R', 'LAND CRUISER', 'MARK X', 'DELICA D:2', '2 СЕРИИ GRAN TOURER', 'SPRINTER TRUENO', 'LANCER EVOLUTION', 'COROLLA SPACIO', 'SL-КЛАСС AMG', 'RAV4', 'BLUEBIRD SYLPHY', 'LITE ACE', 'CR-Z', 'M-КЛАСС AMG', 'PAJERO PININ', 'MARK X ZIO', 'JUKE NISMO', 'DELICA D:5', 'ALMERA CLASSIC', 'RS Q3', 'C-КЛАСС'

In [28]:
train['model'] = train.car_url.apply(lambda q: q.split('/')[7])
test['model'] = test.car_url.apply(lambda q: q.split('/')[7])

In [29]:
vector_items('model')

Statistics of items in vector: model.
 ---------------------------
Train has 528 unique items and has difference: set(),
--
Test has 544 unique items and has difference: {'sera', 'e3', 'will_cypha', '120', 'w188', 'lm', '100_series', 'x3_m', '140', '100nx', '280zx', 'popular', 'origin', 'simplex', 'golf_r32', 'i'}.
{'v40', 'liberty', 'avenir', 'm_klasse', 'simplex', 'qx80', 'tiguan', '460', 'ls', 'g_klasse_amg', 'nx', 'aygo', 'b_klasse', 'grandis', 'curren', '326', 'q', 'golf_r', 'sequoia', 'accord', 'stream', 'carina_e', 'prairie', 'pathfinder', 'x3_m', '100', 'elysion', 'delica_d2', 'caddy', 'verossa', 'z4', 'glb_klasse', 'sienta', 'venza', 'a_klasse_amg', 'sprinter_carib', 'g35', 'roomster', 'gs', 'primera', 'lancer_evolution', 'corolla_spacio', 'yeti', 'w111', 'jx', 'almera_tino', 'verso_s', 'allroad', 'pulsar', 'corolla_levin', 'crosstour', 'w188', 'x7', 'jazz', 'm_klasse_amg', 'bora', 'coupe', 'mr2', 'a6', 'vigor', 'rs3', 'cr_v', 's90', 'porte', 'avalon', 'origin', 'cynos', 'mark

---
### resume:
Vectors 'model_name' and 'name' is very garbage. May be it will be worth to drop them and to use only
vectors 'brand' and 'model'


---
# color

In [30]:
vector_items('color')

Statistics of items in vector: color.
 ---------------------------
Train has 16 unique items and has difference: set(),
--
Test has 16 unique items and has difference: set().
{'золотистый', 'серебристый', 'оранжевый', 'пурпурный', 'голубой', 'зелёный', 'белый', 'бежевый', 'чёрный', 'розовый', 'красный', 'серый', 'фиолетовый', 'синий', 'жёлтый', 'коричневый'}


---
## engineDisplacement, fuelType

In [31]:
vector_items('fuelType')

Statistics of items in vector: fuelType.
 ---------------------------
Train has 9 unique items and has difference: {'Бензин, газобаллонное оборудование', 'Дизель, газобаллонное оборудование', 'Газ', 'Электро', 'Газ, газобаллонное оборудование', 'Гибрид, газобаллонное оборудование', 'Бензин', 'Гибрид', 'Дизель'},
--
Test has 5 unique items and has difference: {'гибрид', 'бензин', 'электро', 'газ', 'дизель'}.
{'гибрид', 'бензин', 'электро', 'газ', 'дизель'}


In [32]:
# replace long name ', газобаллонное оборудование' on '-гбо' and make all letters lower case
train.fuelType = train.fuelType.str.replace(', газобаллонное оборудование', '-гбо')
train.fuelType = train.fuelType.str.lower()

In [33]:
vector_items('fuelType')

Statistics of items in vector: fuelType.
 ---------------------------
Train has 9 unique items and has difference: {'гибрид-гбо', 'дизель-гбо', 'газ-гбо', 'бензин-гбо'},
--
Test has 5 unique items and has difference: set().
{'гибрид', 'бензин', 'электро', 'газ', 'дизель'}


In [34]:
""" function to take only numeric data from string"""
def find_number(field: str):
    """
    :param field: string to find number
    :return: number (in string format)
    """

    # mask for search
    p = '[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+'

    # main cycle for searching
    if re.search(p, field) is not None:
        for catch in re.finditer(p, field):
            return catch[0]

In fact, electro-cars have no engine displacement.
In datasets vector 'engineDisplacement' electro-cars records have different filling.
In test dataset vector 'engineDisplacement' filled with *'LTR'*, in train dataset - they filled with *horse power value*.
in test dataset vector 'enginePower' filled with *horse power value*, in train dataset - with *kVt power value*

We need to make something uniform. So we will make new vector 'engineName' as mask ['fuelType'+' '+'engineDisplacement'] for non-electro and ['fuelType'+' '+ kVt power value] for electro cars.

In [35]:
# make the dict from pairs (horse power, kVt power)
kvt=dict()

qq = train[train.fuelType == 'электро'].groupby(['engineDisplacement', 'enginePower'])['enginePower'].count()
qq = qq.to_dict()
for item, value in qq.keys():
    kvt[int(find_number(item))] = int(find_number(value))
kvt

{109: 80,
 115: 85,
 136: 100,
 150: 110,
 154: 113,
 170: 125,
 179: 132,
 184: 135,
 218: 160,
 313: 230,
 408: 300,
 41: 30,
 64: 47,
 67: 49,
 74: 100}

In [36]:
""" function to make new uniform vector of type of engine as mentioned above"""
def engineDisplacement(record):
    """
    :param record: raw from dataset
    :return: value for new vector
    """

    # case of gasoline and diesel
    if record.fuelType != 'электро':
        result = record.fuelType + ' ' + find_number(record.engineDisplacement)

    # case of electro
    else:
        # case for train dataset, where we have filled vector engineDisplacement, so we can take kVt power direct
        if find_number(record.engineDisplacement):
            result = record.fuelType + ' ' + find_number(record.enginePower)

        # case for test dataset, where we only have horse power filling
        else:
            value_kvt = int(record.enginePower.split()[0])
            try:
                # use dict to find value of kVt power from horse power
                value_kvt = str(kvt[value_kvt])

            # someone we don't have value of kVt power, so we leave just 'электро'
            except:
                value_kvt = ''

            result = record.fuelType + ' ' + str(value_kvt)

    return result

In [37]:
train['engineName'] = train.apply(engineDisplacement, axis=1)
test['engineName'] = test.apply(engineDisplacement, axis=1)

In [38]:
vector_items('engineName')

Statistics of items in vector: engineName.
 ---------------------------
Train has 157 unique items and has difference: {'бензин-гбо 2.7', 'электро 113', 'бензин-гбо 2.9', 'бензин-гбо 6.0', 'гибрид 1.4', 'бензин-гбо 1.6', 'бензин-гбо 2.0', 'бензин-гбо 1.3', 'бензин-гбо 3.5', 'бензин-гбо 3.6', 'гибрид-гбо 1.3', 'бензин-гбо 2.1', 'электро 160', 'бензин-гбо 3.3', 'бензин-гбо 1.8', 'бензин-гбо 4.5', 'электро 230', 'бензин-гбо 4.8', 'бензин-гбо 5.6', 'дизель-гбо 3.0', 'бензин-гбо 3.8', 'дизель-гбо 2.8', 'бензин-гбо 2.4', 'бензин-гбо 5.5', 'бензин-гбо 1.5', 'бензин-гбо 2.6', 'бензин-гбо 5.7', 'гибрид-гбо 3.5', 'газ-гбо 1.4', 'бензин-гбо 3.0', 'бензин-гбо 2.3', 'бензин-гбо 4.0', 'дизель-гбо 4.5', 'газ 2.0', 'бензин-гбо 3.2', 'бензин-гбо 2.8', 'дизель 3.3', 'бензин-гбо 4.7', 'бензин-гбо 1.7', 'бензин-гбо 2.2', 'бензин-гбо 3.4', 'бензин-гбо 3.7', 'газ-гбо 2.0', 'газ-гбо 1.8', 'дизель 1.8', 'электро 30', 'бензин-гбо 4.6', 'бензин-гбо 4.4', 'бензин-гбо 4.2', 'бензин-гбо 4.3', 'бензин-гбо 2.5', 'га

In [39]:
test.enginePower = test.enginePower.str.replace('N12', 'л.с.')

In [40]:
train['enginePowerValue'] = train.enginePower
test['enginePowerValue'] = test.enginePower

In [41]:
def enginePowerExchange(record):
    if record.enginePower.split()[1] == 'кВт':
        result = find_number(record.engineDisplacement)
    else:
        result = find_number(record.enginePower)
    return result

In [42]:
train.enginePower = train.apply(enginePowerExchange, axis=1)

In [43]:
test.enginePower = test.apply(enginePowerExchange, axis=1)

In [44]:
train.enginePower = train.enginePower.apply(int)
test.enginePower = test.enginePower.apply(int)


In [45]:
vector_items('enginePower')

Statistics of items in vector: enginePower.
 ---------------------------
Train has 325 unique items and has difference: {134, 274, 537, 410, 670, 415, 161, 34, 417, 41, 299, 176, 564, 314, 187, 448, 449, 456, 206, 344, 89, 237},
--
Test has 315 unique items and has difference: {32, 195, 164, 550, 487, 457, 42, 44, 430, 214, 30, 127}.
{30, 32, 38, 40, 42, 44, 46, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 60, 61, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 190, 192, 193, 194, 195, 197, 199, 200, 201, 2

In [46]:
vector_items('engineDisplacement')

Statistics of items in vector: engineDisplacement.
 ---------------------------
Train has 68 unique items and has difference: {'5.7 л', '1.5 л', '1.2 л', '2.9 л', '67 л.с.', '2.4 л', '2.3 л', '184 л.с.', '170 л.с.', '4.6 л', '5.0 л', '2.6 л', '1.4 л', '3.0 л', '3.6 л', '3.4 л', '5.2 л', '4.3 л', '6.3 л', '3.1 л', '6.2 л', '4.4 л', '109 л.с.', '1.7 л', '4.7 л', '4.1 л', '4.8 л', '1.3 л', '5.9 л', '4.5 л', '3.2 л', '1.1 л', '74 л.с.', '136 л.с.', '115 л.с.', '1.9 л', '2.8 л', '150 л.с.', '179 л.с.', '3.3 л', '5.5 л', '5.6 л', '1.6 л', '5.4 л', '41 л.с.', '2.0 л', '408 л.с.', '6.0 л', '154 л.с.', '1.8 л', '4.2 л', '3.7 л', '313 л.с.', '0.7 л', '2.1 л', '1.0 л', '3.8 л', '4.0 л', '218 л.с.', '2.2 л', '2.7 л', '3.9 л', '64 л.с.', '6.6 л', '2.5 л', '5.8 л', '4.9 л', '3.5 л'},
--
Test has 55 unique items and has difference: {'3.6 LTR', '6.6 LTR', '1.6 LTR', '4.1 LTR', '4.4 LTR', '5.5 LTR', '3.4 LTR', '4.9 LTR', '1.0 LTR', '2.5 LTR', '2.3 LTR', '1.8 LTR', '3.8 LTR', '5.2 LTR', '2.4 LTR', '4.3 

In [47]:
test.engineDisplacement = test.engineDisplacement.str.replace('LTR', 'л')

In [48]:
vector_items('engineDisplacement')

Statistics of items in vector: engineDisplacement.
 ---------------------------
Train has 68 unique items and has difference: {'67 л.с.', '184 л.с.', '41 л.с.', '170 л.с.', '74 л.с.', '218 л.с.', '64 л.с.', '115 л.с.', '136 л.с.', '313 л.с.', '408 л.с.', '154 л.с.', '150 л.с.', '179 л.с.', '109 л.с.'},
--
Test has 55 unique items and has difference: {'5.3 л', ' л'}.
{'5.7 л', '1.5 л', '1.2 л', '2.9 л', '2.4 л', '2.3 л', '4.6 л', '5.0 л', '2.6 л', '1.4 л', '3.0 л', '3.6 л', '3.4 л', '5.2 л', '4.3 л', '6.3 л', '3.1 л', '6.2 л', '4.4 л', '1.7 л', '4.7 л', '4.1 л', '4.8 л', '1.3 л', '1.1 л', '5.9 л', '3.2 л', '4.5 л', ' л', '1.9 л', '2.8 л', '5.3 л', '3.3 л', '5.5 л', '5.6 л', '1.6 л', '5.4 л', '2.0 л', '6.0 л', '1.8 л', '4.2 л', '3.7 л', '0.7 л', '2.1 л', '1.0 л', '3.8 л', '4.0 л', '2.7 л', '2.2 л', '3.9 л', '6.6 л', '2.5 л', '5.8 л', '4.9 л', '3.5 л'}


---
# vehicleConfiguration

In [49]:
vector_items('vehicleTransmission')

Statistics of items in vector: vehicleTransmission.
 ---------------------------
Train has 4 unique items and has difference: set(),
--
Test has 4 unique items and has difference: set().
{'вариатор', 'механическая', 'автоматическая', 'роботизированная'}


In [50]:
transmission = {
    'механическая': 'MECHANICAL',
    'вариатор': 'VARIATOR',
    'роботизированная': 'ROBOT',
    'автоматическая': 'AUTOMATIC'
}

In [51]:
temp_df = test[
    ['bodyType', 'vehicleConfiguration']].groupby(
    ['bodyType', 'vehicleConfiguration'])['vehicleConfiguration'].count().to_dict()

bodyTypeEng = dict()
for key, value in temp_df.keys():
    bodyTypeEng[key] = value.split()[0]

bodyTypeEng

{'внедорожник 3 дв.': 'ALLROAD_3_DOORS',
 'внедорожник 5 дв.': 'ALLROAD_5_DOORS',
 'внедорожник открытый': 'ALLROAD_OPEN',
 'кабриолет': 'CABRIO',
 'компактвэн': 'COMPACTVAN',
 'купе': 'COUPE',
 'купе-хардтоп': 'COUPE_HARDTOP',
 'лимузин': 'LIMOUSINE',
 'лифтбек': 'LIFTBACK',
 'микровэн': 'MICROVAN',
 'минивэн': 'MINIVAN',
 'пикап двойная кабина': 'PICKUP_TWO',
 'пикап одинарная кабина': 'PICKUP_ONE',
 'пикап полуторная кабина': 'PICKUP_ONE_HALF',
 'родстер': 'ROADSTER',
 'седан': 'SEDAN',
 'седан 2 дв.': 'SEDAN_2_DOORS',
 'седан-хардтоп': 'SEDAN_HARDTOP',
 'тарга': 'TARGA',
 'универсал 5 дв.': 'WAGON_5_DOORS',
 'фастбек': 'FASTBACK',
 'фургон': 'VAN',
 'хэтчбек 3 дв.': 'HATCHBACK_3_DOORS',
 'хэтчбек 5 дв.': 'HATCHBACK_5_DOORS'}

In [52]:
def vehicleConfiguration(record):
    body = bodyTypeEng[record.bodyType]
    trans = transmission[record.vehicleTransmission]
    disp = find_number(record.engineDisplacement)

    result = body + ' ' + trans
    if disp:
        result = result + ' ' + disp

    return result

In [53]:
train.vehicleConfiguration = train.apply(vehicleConfiguration, axis=1)

In [54]:
vector_items('vehicleConfiguration')

Statistics of items in vector: vehicleConfiguration.
 ---------------------------
Train has 755 unique items and has difference: {'COUPE MECHANICAL 4.8', 'HATCHBACK_3_DOORS ROBOT 1.0', 'HATCHBACK_5_DOORS AUTOMATIC 67', 'ALLROAD_5_DOORS MECHANICAL 2.9', 'CABRIO MECHANICAL 1.5', 'COUPE MECHANICAL 2.1', 'LIFTBACK ROBOT 1.5', 'VAN VARIATOR 2.0', 'WAGON_5_DOORS AUTOMATIC 1.2', 'ROADSTER MECHANICAL 2.0', 'CABRIO MECHANICAL 4.0', 'CABRIO MECHANICAL 2.3', 'PICKUP_ONE AUTOMATIC 2.5', 'CABRIO AUTOMATIC 3.2', 'CABRIO AUTOMATIC 2.1', 'VAN MECHANICAL 1.2', 'COUPE MECHANICAL 1.2', 'PICKUP_ONE_HALF MECHANICAL 2.0', 'COMPACTVAN AUTOMATIC 1.9', 'SEDAN AUTOMATIC 109', 'VAN AUTOMATIC 0.7', 'SEDAN_HARDTOP AUTOMATIC 1.5', 'SEDAN_HARDTOP MECHANICAL 1.6', 'HATCHBACK_3_DOORS MECHANICAL 3.0', 'ALLROAD_3_DOORS MECHANICAL 0.7', 'SEDAN_2_DOORS MECHANICAL 1.8', 'SEDAN VARIATOR 2.7', 'HATCHBACK_5_DOORS MECHANICAL 2.8', 'ROADSTER ROBOT 3.2', 'PICKUP_TWO AUTOMATIC 4.0', 'MICROVAN AUTOMATIC 41', 'ALLROAD_5_DOORS AUTOM

In [55]:
train.drop(['modelDate', 'model_info', 'model_name', 'name', 'price_current'], axis=1, inplace=True)
test.drop(['modelDate', 'model_info', 'model_name', 'name'], axis=1, inplace=True)

---
# vendor

In [56]:
vend = test.groupby(['brand', 'vendor'])['vendor'].count().to_dict().keys()
vend=dict(vend)

In [57]:
train.vendor = train.brand.map(vend)

In [58]:
vector_items('vendor')

Statistics of items in vector: vendor.
 ---------------------------
Train has 2 unique items and has difference: set(),
--
Test has 2 unique items and has difference: set().
{'EUROPEAN', 'JAPANESE'}


---
# Владение

In [59]:
vector_items('Владельцы')

Statistics of items in vector: Владельцы.
 ---------------------------
Train has 4 unique items and has difference: {'1 владелец', '2 владельца', nan},
--
Test has 3 unique items and has difference: {'2\xa0владельца', '1\xa0владелец'}.
{'2\xa0владельца', '3 или более', '1\xa0владелец'}


In [60]:
train['Владельцы'].fillna('3 или более', inplace=True)

In [61]:
test['owners'] = test['Владельцы']
test['Владельцы'] = test['Владельцы'].apply(find_number)

In [62]:
train['owners'] = train['Владельцы']
train['Владельцы'] = train['Владельцы'].apply(find_number)

In [63]:
train['Владельцы'] = train['Владельцы'].apply(int)
test['Владельцы'] = test['Владельцы'].apply(int)

In [64]:
vector_items('Владельцы')

Statistics of items in vector: Владельцы.
 ---------------------------
Train has 3 unique items and has difference: set(),
--
Test has 3 unique items and has difference: set().
{1, 2, 3}


In [65]:
train.drop(['Владение'], axis=1, inplace=True)
test.drop(['Владение'], axis=1, inplace=True)

---
# ПТС

In [66]:
test['ПТС'].value_counts()

Оригинал    30098
Дубликат     4587
Name: ПТС, dtype: int64

In [67]:
train['ПТС'].fillna('Дубликат', inplace=True)
test['ПТС'].fillna('Дубликат', inplace=True)

In [68]:
vector_items('ПТС')

Statistics of items in vector: ПТС.
 ---------------------------
Train has 2 unique items and has difference: set(),
--
Test has 2 unique items and has difference: set().
{'Оригинал', 'Дубликат'}


---
# Состояние

In [69]:
vector_items('Состояние')

Statistics of items in vector: Состояние.
 ---------------------------
Train has 2 unique items and has difference: {nan},
--
Test has 1 unique items and has difference: set().
{'Не требует ремонта'}


In [70]:
train[train['Состояние'] != 'Не требует ремонта']

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,ПТС,Привод,Руль,Состояние,Таможня,price,model,engineName,enginePowerValue,owners
36691,внедорожник 5 дв.,MERCEDES,https://auto.ru/cars/used/sale/mercedes/glb_kl...,белый,"{'back-brake': 'Disc', 'feeding': 'Turbocharge...",Куплен 25 декабря 2020 у официального дилера в...,1.3 л,150,{},бензин,...,Оригинал,передний,Левый,NaN,Растаможен,2.229825e+06,glb_klasse,бензин 1.3,150 л.с.,1 владелец
67936,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,серебристый,"{'back-brake': 'Disc', 'feeding': 'None', 'ful...",Битая Цена ремонта на 150 тр,1.6 л,102,"{'cruise-control': True, 'tinted-glass': True,...",бензин,...,Оригинал,передний,Левый,NaN,Растаможен,1.664049e+05,octavia,бензин 1.6,102 л.с.,3 или более
77393,седан,TOYOTA,https://auto.ru/cars/used/sale/toyota/corolla/...,серый,"{'feeding': 'None', 'horse-power': 105, 'kvt-p...",Надо Делать ДВС Обмен Торг,1.6 л,105,{},бензин,...,Оригинал,передний,Правый,NaN,Растаможен,8.320243e+04,corolla,бензин 1.6,105 л.с.,2 владельца
77431,седан,TOYOTA,https://auto.ru/cars/used/sale/toyota/corolla/...,белый,"{'back-brake': 'Drum', 'feeding': 'None', 'ran...",продается легенда японского автопрома битая по...,1.6 л,115,{},бензин,...,Дубликат,передний,Правый,NaN,Растаможен,6.240182e+04,corolla,бензин 1.6,115 л.с.,3 или более
89883,хэтчбек 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/golf...,красный,"{'back-brake': 'Drum', 'feeding': 'None', 'ful...",Проблемы с электрикой производилась замена вых...,1.6 л,75,"{'ptf': True, 'abs': True, 'eco-leather': True...",бензин,...,Дубликат,передний,Левый,NaN,Растаможен,1.248036e+05,golf,бензин 1.6,75 л.с.,3 или более
89888,хэтчбек 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/golf...,чёрный,"{'back-brake': 'Drum', 'feeding': 'None', 'ful...",Замена порогов задняя юбка днище обработана ан...,1.6 л,72,{},бензин,...,Оригинал,передний,Левый,NaN,Растаможен,1.331239e+05,golf,бензин 1.6,72 л.с.,3 или более
94015,универсал 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/pass...,белый,"{'back-brake': 'Drum', 'feeding': 'None', 'ful...",На разбор или под восстановление без АКБ Торг ...,1.8 л,107,{},бензин,...,Оригинал,передний,Левый,NaN,Растаможен,5.408158e+04,passat,бензин 1.8,107 л.с.,3 или более


In [71]:
train.dropna(subset=['Состояние'], inplace=True)

In [72]:
vector_items('Состояние')

Statistics of items in vector: Состояние.
 ---------------------------
Train has 1 unique items and has difference: set(),
--
Test has 1 unique items and has difference: set().
{'Не требует ремонта'}


In [73]:
train.drop(['Состояние'], axis=1, inplace=True)
test.drop(['Состояние'], axis=1, inplace=True)

---
# Таможня

In [74]:
vector_items('Таможня')

Statistics of items in vector: Таможня.
 ---------------------------
Train has 2 unique items and has difference: {nan},
--
Test has 1 unique items and has difference: set().
{'Растаможен'}


In [75]:
train.dropna(subset=['Таможня'], inplace=True)

In [76]:
vector_items('Таможня')

Statistics of items in vector: Таможня.
 ---------------------------
Train has 1 unique items and has difference: set(),
--
Test has 1 unique items and has difference: set().
{'Растаможен'}


In [77]:
train.drop(['Таможня'], axis=1, inplace=True)
test.drop(['Таможня'], axis=1, inplace=True)

---
# Prepare for Feature engineering

In [86]:
train.drop(['complectation_dict', 'equipment_dict',
            'image', 'description', 'priceCurrency'], axis=1, inplace=True)

In [93]:
test.drop(['complectation_dict', 'equipment_dict',
           'image', 'description', 'priceCurrency'], axis=1, inplace=True)

In [88]:
test['price']=0
test['test']=1
train['test']=0

In [94]:
data = train.append(test)

In [95]:
data.shape, train.shape, test.shape

((122432, 26), (87746, 26), (34686, 26))

In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122432 entries, 0 to 34685
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   bodyType              122432 non-null  object 
 1   brand                 122432 non-null  object 
 2   car_url               122432 non-null  object 
 3   color                 122432 non-null  object 
 4   engineDisplacement    122432 non-null  object 
 5   enginePower           122432 non-null  int64  
 6   fuelType              122432 non-null  object 
 7   mileage               122432 non-null  int64  
 8   numberOfDoors         122432 non-null  int64  
 9   parsing_unixtime      122432 non-null  int64  
 10  productionDate        122432 non-null  int64  
 11  sell_id               122432 non-null  int64  
 12  super_gen             122432 non-null  object 
 13  vehicleConfiguration  122432 non-null  object 
 14  vehicleTransmission   122432 non-null  object 
 15  v

---
# FE

* average mileage per year
* average mileage by owner
* rarity
* older 3-5 year
* color clustering
* body clustering
*

In [87]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87746 entries, 0 to 103793
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              87746 non-null  object 
 1   brand                 87746 non-null  object 
 2   car_url               87746 non-null  object 
 3   color                 87746 non-null  object 
 4   engineDisplacement    87746 non-null  object 
 5   enginePower           87746 non-null  int64  
 6   fuelType              87746 non-null  object 
 7   mileage               87746 non-null  int64  
 8   numberOfDoors         87746 non-null  int64  
 9   parsing_unixtime      87746 non-null  int64  
 10  productionDate        87746 non-null  int64  
 11  sell_id               87746 non-null  int64  
 12  super_gen             87746 non-null  object 
 13  vehicleConfiguration  87746 non-null  object 
 14  vehicleTransmission   87746 non-null  object 
 15  vendor            

In [79]:
test.isnull().sum()

bodyType                    0
brand                       0
car_url                     0
color                       0
complectation_dict      28268
description                 0
engineDisplacement          0
enginePower                 0
equipment_dict           9996
fuelType                    0
image                       0
mileage                     0
numberOfDoors               0
parsing_unixtime            0
priceCurrency               0
productionDate              0
sell_id                     0
super_gen                   0
vehicleConfiguration        0
vehicleTransmission         0
vendor                      0
Владельцы                   0
ПТС                         0
Привод                      0
Руль                        0
model                       0
engineName                  0
enginePowerValue            0
owners                      0
dtype: int64

In [84]:
train.isnull().sum()

bodyType                0
brand                   0
car_url                 0
color                   0
complectation_dict      0
description             0
engineDisplacement      0
enginePower             0
equipment_dict          0
fuelType                0
image                   1
mileage                 0
numberOfDoors           0
parsing_unixtime        0
priceCurrency           0
productionDate          0
sell_id                 0
super_gen               0
vehicleConfiguration    0
vehicleTransmission     0
vendor                  0
Владельцы               0
ПТС                     0
Привод                  0
Руль                    0
price                   0
model                   0
engineName              0
enginePowerValue        0
owners                  0
dtype: int64

----
# CatBosting

In [ ]:
features = ['bodyType', 'brand', 'color', 'engineDisplacement',
            'enginePower', 'fuelType', 'mileage',
            'numberOfDoors', 'productionDate', 'vehicleTransmission']
features_obj =['bodyType', 'brand', 'color', 'engineDisplacement',
               'enginePower', 'fuelType',
               'numberOfDoors', 'vehicleTransmission']
train_f = train[features].copy()
for colum in features_obj:
    train_f[colum] = train_f[colum].astype('category').cat.codes

In [ ]:
train_f

In [ ]:
target = 'price'
#x = train.drop(['price', 'model_info', 'modelDate', 'vehicleConfiguration',
#                'vendor', 'Владельцы', 'Владение' ], axis=1)
x = train_f
y = train[target]

In [ ]:
stack = x.copy()
stack['price']=y
stack.corr()['price'].abs().sort_values(ascending=False)[1:]

In [ ]:
heatmap([],stack)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=SEED)

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                          )
model.fit(X_train, y_train,
          #cat_features=cat_features_ids,
          eval_set=(X_test, y_test),
          verbose_eval=0,
          use_best_model=True,
          #plot=True
          )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                          )
model.fit(X_train, np.log(y_train),
          #cat_features=cat_features_ids,
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=0,
          use_best_model=True,
          #plot=True
          )

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
predict_test = np.exp(model.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

---
# Test various models

In [ ]:
test_classifires = {
    'RandomForest': RandomForestRegressor(n_estimators=300, random_state = SEED),
    'DecisionTree': DecisionTreeRegressor(random_state=SEED),
    'ExtraTrees'  : ExtraTreesRegressor(random_state=SEED),
    'LightGBM'    : LGBMRegressor(random_state=SEED),
    'XGBoost'     : XGBRegressor(random_state=SEED)
}

In [ ]:
results = dict()
for item in tqdm(test_classifires):
    cls=test_classifires[item]

    cls.fit(X_train, y_train)
    predict =cls.predict(X_test)

    cls.fit(X_train, np.log(y_train))
    predict_log = np.exp(cls.predict(X_test))

    results[item] = [round(mape(y_test, predict)*100,2), round(mape(y_test, predict_log)*100,2)]
results

---
# RandomForest best parameters

In [ ]:
# Making best parameters searching diapasons
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}

# Randomized search. Making 3 cross validation (cv) for each 100 iterations candidates
model_rfc = RandomForestRegressor(random_state = SEED)
rs = RandomizedSearchCV(model_rfc,
                        param_dist,
                        n_iter = 100,
                        cv = 3,
                        verbose = 1,
                        n_jobs=-1,
                        random_state=SEED)
rs.fit(X_train, np.log(y_train))
rfc_best = rs.best_params_
rfc_best['random_state']=SEED
rfc_best
# approx. 16 minutes
# {'n_estimators': 200,
# 'min_samples_split': 2,
# 'min_samples_leaf': 2,
# 'max_features': 'sqrt',
# 'max_depth': 14,
# 'bootstrap': False,
# 'random_state': 42}

In [ ]:
#
model_rfc = RandomForestRegressor(**rfc_best)

model_rfc.fit(X_train, np.log(y_train))
predict_log = np.exp(model_rfc.predict(X_test))
round(mape(y_test, predict_log)*100,2)

In [ ]:
rs_df = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_df = rs_df.drop([
    'mean_fit_time',
    'std_fit_time',
    'mean_score_time',
    'std_score_time',
    'params',
    'split0_test_score',
    'split1_test_score',
    'split2_test_score',
    'std_test_score'],
    axis=1)

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2)
sns.set(style="whitegrid", color_codes=True, font_scale = 2)
fig.set_size_inches(30,25)
sns.barplot(x='param_n_estimators', y='mean_test_score', data=rs_df, ax=axs[0,0], color='lightgrey')
axs[0,0].set_ylim([.83,.95])
axs[0,0].set_title(label = 'n_estimators', size=30, weight='bold')
sns.barplot(x='param_min_samples_split', y='mean_test_score', data=rs_df, ax=axs[0,1], color='coral')
axs[0,1].set_ylim([.85,.95])
axs[0,1].set_title(label = 'min_samples_split', size=30, weight='bold')
sns.barplot(x='param_min_samples_leaf', y='mean_test_score', data=rs_df, ax=axs[0,2], color='lightgreen')
axs[0,2].set_ylim([.80,.95])
axs[0,2].set_title(label = 'min_samples_leaf', size=30, weight='bold')
sns.barplot(x='param_max_features', y='mean_test_score', data=rs_df, ax=axs[1,0], color='wheat')
axs[1,0].set_ylim([.80,.95])
axs[1,0].set_title(label = 'max_features', size=30, weight='bold')
sns.barplot(x='param_max_depth', y='mean_test_score', data=rs_df, ax=axs[1,1], color='lightpink')
axs[1,1].set_ylim([.80,.95])
axs[1,1].set_title(label = 'max_depth', size=30, weight='bold')
sns.barplot(x='param_bootstrap',y='mean_test_score', data=rs_df, ax=axs[1,2], color='skyblue')
axs[1,2].set_ylim([.80,.95])
axs[1,2].set_title(label = 'bootstrap', size=30, weight='bold')
plt.show()

---
# DecisionTree

---
# GradientBoosting

---
# ExtraTrees

---
# LightGBM

---
# XGBoost